In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

# Load Foil Data

In [ ]:
from ICARUS.Computation.Solvers.XFLR5.polars import read_polars_2d
from ICARUS.Database import DB
from ICARUS.Core.struct import Struct
from ICARUS.Database import EXTERNAL_DB

# Get Plane

In [ ]:
# from Vehicles.Planes.hermes import hermes
# airplane = hermes("Hermes")

# from ICARUS.Computation.Solvers.XFLR5.parser import parse_xfl_project
# filename: str = f"{EXTERNAL_DB}/plane_1.xml"
# airplane = parse_xfl_project(filename)
# airplane.name = "Embraer_E190_Cruise"
# airplane.visualize()


# airplane = DB.vehicles_db.planes['hermes_3']
# airplane.save()

In [ ]:
from Vehicles.Planes.hermes import hermes
airplane = hermes("hermes")

# Import Environment and Set State

In [ ]:
from ICARUS.Environment.definition import EARTH_ISA
from ICARUS.Flight_Dynamics.state import State

u_freestream = 20
unstick = State(
    name= "unstick",
    airplane= airplane,
    environment= EARTH_ISA,
    u_freestream= u_freestream
)

# Get Solver

In [ ]:
from ICARUS.Computation.Solvers.GenuVP.gnvp3 import GenuVP3
gnvp3 = GenuVP3()

from ICARUS.Computation.Solvers.GenuVP.gnvp7 import GenuVP7
gnvp7 = GenuVP7()

from ICARUS.Computation.Solvers.Icarus_LSPT.wing_lspt import LSPT
lspt = LSPT()

# print(lspt)
print(gnvp3)
# print(gnvp7)

## AoA Run

In [ ]:
analysis: str = gnvp3.get_analyses_names()[0]
print(f"Selecting Analysis: {analysis}")
gnvp3.select_analysis(analysis)

In [ ]:
options: Struct = gnvp3.get_analysis_options(verbose=True)
solver_parameters: Struct = gnvp3.get_solver_parameters(verbose=True)

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype= float)
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane         = airplane
options.state         = unstick
options.solver2D      = 'Xfoil'
options.maxiter       = 100
options.timestep      = 0.001
options.angles        = angles

solver_parameters.Use_Grid = 0

gnvp3.define_analysis(options, solver_parameters)
gnvp3.print_analysis_options()
gnvp3.execute(parallel = False)
polars = gnvp3.get_results()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.Flight_Dynamics.state import State
if isinstance(polars,int):
    raise Exception("No Polars")
unstick = State("Unstick",airplane,environment= EARTH_ISA, u_freestream= u_freestream)
unstick.add_polar(polars)

### Pertrubations

In [ ]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

In [ ]:
analysis = gnvp3.get_analyses_names(verbose=True)[1] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.select_analysis(analysis)
options = gnvp3.get_analysis_options(verbose=True)

In [31]:
options.plane         = airplane
options.state         = unstick
options.solver2D      = 'XFLR'
options.maxiter       = 200
options.timestep      = 5e-3

gnvp3.print_analysis_options()
gnvp3.define_analysis(options, solver_parameters)
gnvp3.execute(parallel=True)
gnvp3.get_results()

Analysis Completed
Getting Results
Logging Pertrubations


,Type,Epsilon,GenuVP3 Potential Fx,GenuVP3 Potential Fy,GenuVP3 Potential Fz,GenuVP3 Potential Mx,GenuVP3 Potential My,GenuVP3 Potential Mz,GenuVP3 2D Fx,GenuVP3 2D Fy,...,GenuVP3 ONERA Mx,GenuVP3 ONERA My,GenuVP3 ONERA Mz,AoA,Fx,Fy,Fz,Mx,My,Mz
0,Trim,0.00,-160.850326,1.842814,183.815362,184.186523,0.754482,166.886398,-40.250177,0.000000,...,28.492839,0.105116,30.398067,-4.229261,-40.250177,0.000000,42.999442,19.709573,-0.010957,20.371937
1,p,0.25,-160.946594,1.920734,184.070103,184.240294,0.670265,166.903696,-40.266294,0.000000,...,28.546287,0.126016,30.437089,-4.229261,-40.266294,0.000000,42.941768,19.730181,0.012514,20.379412
2,p,-0.25,-161.194517,1.827675,184.040503,184.368719,0.735081,167.157628,-40.286499,0.000000,...,28.452761,0.120445,30.383179,-4.229261,-40.286499,0.000000,42.916661,19.706022,0.007007,20.391187
3,phi,-0.01,-161.247728,1.864814,184.114824,184.352671,0.719706,167.102175,-40.294463,0.008452,...,28.475007,0.095358,30.396761,-4.229261,-40.294463,0.008452,42.965285,19.711279,-0.002878,20.388489
4,phi,0.01,-160.974014,1.906901,183.978129,184.220076,0.671832,167.001647,-40.257708,-0.008446,...,28.487048,0.116768,30.396501,-4.229261,-40.257708,-0.008446,42.936311,19.710148,0.001502,20.376934
5,q,-0.25,-161.026901,1.893928,183.984873,184.254762,0.709123,167.027015,-40.262621,0.000000,...,28.478606,0.112657,30.392376,-4.229261,-40.262621,0.000000,42.929877,19.708288,0.004322,20.379565
6,q,0.25,-161.216638,1.839722,184.093667,184.325435,0.692411,167.098367,-40.282766,0.000000,...,28.459602,0.095981,30.384196,-4.229261,-40.282766,0.000000,42.916842,19.704487,0.004633,20.385134
7,r,-0.25,-161.123291,1.904561,184.023293,184.191852,0.669054,167.065733,-40.260618,0.000000,...,28.415681,0.117373,30.345328,-4.229261,-40.260618,0.000000,42.900744,19.684253,0.000685,20.370548
8,r,0.25,-161.052491,1.863440,183.966327,184.312354,0.735078,167.036358,-40.277484,0.000000,...,28.500630,0.109530,30.412713,-4.229261,-40.277484,0.000000,42.973595,19.719622,-0.004647,20.387277
9,theta,0.01,-161.231033,1.765007,184.109977,184.377273,0.710110,167.047418,-40.277459,0.000000,...,28.463658,0.069788,30.379197,-4.229261,-40.277459,0.000000,42.925371,19.706730,0.003627,20.381864


# Sensitivity Analysis

In [ ]:
# if sensAnalysisGNVP3 :
#     for var in ['u','w','q','theta','v','p','r','phi']:
#         space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
#         space =[*-space,*space]
#         maxiter = 2
#         timestep = 5e-2
#         dyn.sensitivityAnalysis(var, space)
#         GNVP3BatchArgs = [dyn, var, foildb, "Xfoil",
#                         maxiter, timestep,
#                         dyn.trim['U'], dyn.trim['AoA']]
#         dyn.executeAnalysis(runGNVPsensitivityParallel, GNVP3BatchArgs)
#         perDIR = os.path.join(dyn.CASEDIR, f"Sensitivity_{var}")
#         dyn.sensResults[var] = pertrResults(perDIR, HOMEDIR)
#     dyn.save()